In [ ]:
import numpy as np
import pandas as pd

import itertools
import random

from sklearn.utils import check_random_state

In [ ]:
RNG_SEED = 42
rng = check_random_state(RNG_SEED)
print(rng)

# Kendall

In [ ]:
a = [[1,4,3,2], [1,1,1,1], [1,1,2,2], [1,2,2,2], [1,1,2,2], [1,2,3,1], [1,3,2,1], [1,1,2,3],
    [1,2,3,4], [1,1,2,1]]
b = [[1,4,3,2], [1,1,1,1], [1,1,2,2], [1,2,2,2], [2,2,1,1], [1,2,3,1], [1,3,2,1], [1,1,2,3],
    [1,1,3,3], [1,1,1,1]]

In [ ]:
import pandas as pd
from sharp.utils import scores_to_ordering
from scipy import stats

def kendall_agreement(results1, results2):
    return results1.reset_index(drop=True).apply(
        lambda row: row_wise_kendall(row, results2.iloc[row.name]), axis=1
    )

def row_wise_kendall(results1, results2):
    results = [results1, results2]

    print("Item1: ", results1.values)
    print("Item2: ", results2.values)
    # Check for ties
    ranks = []
    for result in results:
        rank = scores_to_ordering(result, direction=1)
        # Correct rank values to reflect ties
        for val in result:
            mask = result == val
            if mask.sum() > 1:
                rank[mask] = rank[mask].max()
        ranks.append(rank)

    row_sensitivity = kendall_similarity(*ranks)
    # row_sensitivity = kendall_similarity(*results)
    return row_sensitivity

# def kendall_similarity(a, b):
#     normalizer = (len(a) * (len(a) - 1)) / 2
#     idx_pair = list(combinations(range(len(a)), 2))
#     val_pair_a = [(a[i], a[j]) for i, j in idx_pair if a[i] != a[j]]
#     val_pair_b = [(b[i], b[j]) for i, j in idx_pair if b[i] != b[j]]
#     inversions = sum([(val2, val1) in val_pair_b for val1, val2 in val_pair_a])
#     return (normalizer - inversions) / normalizer

def kendall_similarity(a, b):
    # return 1 - (stats.kendalltau(a, b).statistic + 1) / 2
    print("Rank1: ",a)
    print("Rank2: ",b)
    
    stat = stats.kendalltau(a, b).statistic
    print("Kt-o:", stat)
    print("Kt-n:", 1 - (stat + 1) / 2)
    print("Kt-l", (stat * (-1) + 1) / 2)
    # return stat
    return 1 - (stat + 1) / 2


In [ ]:
# a = [[-10,-6,-4,4,6,10],[-10,-10,-10,0,0,0],[-10,-10,-10,0,0,0],[-10,-8,-6,0,6,8], [-10,-10,-10,0,0,0],[-10,-10,-10,0,0,0],
#      [-10,-10,-10,0,0,0]]
# b = [[-10,-6,-4,4,6,10],[-10,-10,-10,0,0,0],[-10,-10,-10,0,0,-10],[8,6,0,-6,-8,-10], [-10,-9,-8,0,1,2],[-10,-10,-9,0,1,0],
#      [-10,-10,-10,0,0,1]]

kendall_agreement(pd.DataFrame(a), pd.DataFrame(b))

In [ ]:
import pandas as pd
from sharp.utils import scores_to_ordering
from itertools import product, combinations

def kendall_agreement(results1, results2):
    return results1.reset_index(drop=True).apply(
        lambda row: row_wise_kendall(row, results2.iloc[row.name]), axis=1
    )

def row_wise_kendall(results1, results2):
    results = [results1, results2]

    print("Item1: ", results1.values)
    print("Item2: ", results2.values)
    # Check for ties
    ranks = []
    for result in results:
        rank = scores_to_ordering(result, direction=1)
        # Correct rank values to reflect ties
        for val in result:
            mask = result == val
            if mask.sum() > 1:
                rank[mask] = rank[mask].max()
        ranks.append(rank)

    row_sensitivity = kendall_similarity(*ranks)
    # row_sensitivity = kendall_similarity(*results)
    return row_sensitivity

def kendall_similarity(a, b):
    print("Rank1: ",a)
    print("Rank2: ",b)
    normalizer = (len(a) * (len(a) - 1)) / 2
    idx_pair = list(combinations(range(len(a)), 2))
    val_pair_a = [(a[i], a[j]) for i, j in idx_pair if a[i] != a[j]]
    val_pair_b = [(b[i], b[j]) for i, j in idx_pair if b[i] != b[j]]
    print(val_pair_a)
    print(val_pair_b)
    inversions=0
    for (val11, val12), (val21, val22) in zip(val_pair_a, val_pair_b):
        if ((val11 > val12) and (val21 < val22)) or ((val11 < val12) and (val21 > val22)):
            inversions = inversions+1
    kt = 1 - (2 * inversions) / normalizer
    print("Kt-o:", kt)
    print("Kt-n:", 1 - (kt + 1) / 2)
    return 1 - (kt + 1) / 2

In [ ]:
kendall_agreement(pd.DataFrame(a), pd.DataFrame(b))

# Jaccard

In [ ]:
from itertools import product, combinations
from sharp.utils import scores_to_ordering

a = [[3.2, -5.1, 4, 0],
    [3.2, -5.1, 4, 0],
    [3.2, -5.1, 4, 0],
    [3.2, -5.1, 4, 0],
    [3.2, 3.2, 3.2, 3.2]]

b = [[3.9, -3.1, 4.1, 0],
    [3.2, -5.1, 3.2, 0],
    [4, -5.1, 3.2, 0],
    [4, -5.1, 3.2, 3.2],
    [3, 3, 3, 3]]


def _get_importance_mask(row_cont, threshold):
    if threshold >= 1:
        # Calculate order of absolute contributions
        row_abs = np.abs(row_cont)
        print(row_abs)
        # Find n=threshold largest items
        res = sorted(row_abs.index.values, key = lambda sub: row_abs[sub])[-threshold:]
        # Set mask
        mask = pd.Series(data=[True if i in res else False for i in row_cont.index.values],
                         index=row_cont.index.values)
    else:
        # Calculate cumulative absolute contribution order
        total_contribution = np.sum(np.abs(row_cont))
        order = np.argsort(np.abs(row_cont))
        cumulative_cont = np.cumsum(np.abs(row_cont)[order]) / total_contribution
        # Find elements withe the smallest contribution
        # (to meet the threshold it's easier to do the reverse operation)
        mask = (cumulative_cont < 1 - threshold)[order]
        # Reverse array
        mask = ~mask

    print(mask)

    # Check whether ties exist
    possible_configs = [mask.copy()]
    tie_values = [
        (idx_old, cont)
        for idx_old, cont in row_cont[mask].items()
        if cont in row_cont[~mask].values
    ]
    print(tie_values)
    # Make all possible sets of ties
    for idx_old, tie_val in tie_values:
        # idx_new = np.where(row_cont == tie_val)[0]
        idx_new = row_cont[row_cont == tie_val].index.values
        print("Substituting:", idx_new )
        # Exclude all selected indexes that have the same value
        idx_new = list(set(idx_new).difference(mask[mask].index.values))
        print(list(set(idx_new)))
        print("Substituting 2:", idx_new )
        for idx in idx_new:
            new_mask = mask.copy()
            new_mask[idx_old] = False
            new_mask[idx] = True
            possible_configs.append(new_mask)

    print("Possible configs:")
    print(possible_configs)
    return possible_configs

def jaccard_similarity(a, b):
    intersection = len(list(set(a).intersection(b)))
    union = (len(set(a)) + len(set(b))) - intersection
    return float(intersection) / union

def row_wise_jaccard(results1, results2, n_features):
    """
    Calculate the row-wise Jaccard similarity between two sets of results.

    Parameters
    ----------
    results1 : numpy.ndarray
        The first set of results. It should be a 2-dimensional array with shape
        (n_samples, n_features).
    results2 : numpy.ndarray
        The second set of results. It should be a 2-dimensional array with shape
        (n_samples, n_features).
    n_features : int, float or None, default=0.8
        The number of top features to consider. If None, all features are
        considered. If an integer value is provided, only the top n_features
        features are considered. If n_features < 1, the most
        important features are determined based on their contribution to the
        total score (as a percentage of the total contribution in absolute
        values).

    Returns
    -------
    float
        The row-wise Jaccard similarity between the two sets of results.

    Notes
    -----
    The row-wise Jaccard similarity is calculated by first converting the
    results into rankings using the `scores_to_ordering` function. Then, the top
    n_features features are selected based on the rankings. Finally, the Jaccard
    similarity is calculated between the selected features for each row.

    If n_features is less than 1, the most important features are determined
    based on their contribution to the total score.  The cumulative contribution
    of each feature is calculated and the features are selected until the
    cumulative contribution exceeds 1 - n_features.

    Examples
    --------
    >>> results1 = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]])
    >>> results2 = np.array([[0.2, 0.3, 0.4], [0.5, 0.6, 0.7]])
    >>> n_features = 2
    >>> row_wise_jaccard(results1, results2, n_features)
    """

    print("##############")
    print(results1)
    print(results2)
    
    if n_features is None:
        n_features = results1.shape[1]

    masks1 = _get_importance_mask(results1, n_features)
    masks2 = _get_importance_mask(results2, n_features)

    row_similarities = []
    for mask1, mask2 in product(masks1, masks2):
        print("Checking")
        top_idx1 = mask1[mask1].index.values
        top_idx2 = mask2[mask2].index.values
        print(top_idx1)
        print(top_idx2)
        row_similarity = jaccard_similarity(top_idx1, top_idx2)
        row_similarities.append(row_similarity)
    return min(row_similarities)

def jaccard_agreement(results1, results2, n_features=0.8):
    """
    Calculate the Jaccard similarity between two sets of results. Results are
    normalized, 0 means most dis-similar and 1 means most similar.

    Parameters
    ----------
    results1 : pandas.DataFrame
        The first set of results.
    results2 : pandas.DataFrame
        The second set of results.
    n_features : int, float or None, default=0.8
        The number of top features to consider. If None, all features are
        considered. If an integer value is provided, only the top n_features
        features are considered. If n_features < 1, the most
        important features are determined based on their contribution to the
        total score (as a percentage of the total contribution in absolute
        values).

    Returns
    -------
    pandas.Series
        The Jaccard agreement between each pair of results.

    Notes
    -----
    The Jaccard agreement is a measure of similarity between two sets of
    results. It is calculated as the average Jaccard similarity coefficient
    between each pair of results.
    """
    if n_features is None:
        n_features = results1.shape[1]

    return results1.reset_index(drop=True).apply(
        lambda row: row_wise_jaccard(row, results2.iloc[row.name], n_features),
        axis=1,
    )

In [ ]:
a = [[-0.428721,-2.801417,6.815626,-0.092408,-1.213119], [-0.428721,-2.801417,6.815626,-0.092408,-2.801417],
     [-2.801417,-2.801417,6.815626,-0.092408,-2.801417]]

b = [[-0.714536,-4.669028,11.359376,-0.154013,-2.021865], [-0.428721,-2.801417,6.815626,-0.092408,-2.801417],
     [-2.801417,-2.801417,6.815626,-0.092408,-2.801417]]

columns=['teaching', 'research', 'citations', 'income', 'international']

jaccard_agreement(pd.DataFrame(a, columns=columns), pd.DataFrame(b, columns=columns), n_features=0.7)

# Euclidean

In [8]:
import os
import sys

sys.path.append("..")

import tqdm
import time
from copy import deepcopy
import numpy as np
import pandas as pd

import itertools
import random

# from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr

from sklearn.utils import check_random_state
from sklearn.preprocessing import normalize
from lightgbm import LGBMRanker
from sharp import ShaRP
from sharp.utils import scores_to_ordering
from xai_ranking.preprocessing import preprocess_higher_education_data
from xai_ranking.scorers import higher_education_score
from mlresearch.utils import check_random_states

from xai_ranking.benchmarks import (
    human_in_the_loop_experiment,
    hierarchical_ranking_explanation,
    lime_experiment,
    shap_experiment,
    sharp_experiment,
    # rank_lime_experiment,
    # participation_experiment,
)

from xai_ranking.preprocessing import (
    preprocess_atp_data,
    preprocess_csrank_data,
    preprocess_higher_education_data,
    preprocess_movers_data,
    preprocess_synthetic_data,
)
from xai_ranking.datasets import (
    fetch_atp_data,
    fetch_csrank_data,
    fetch_higher_education_data,
    fetch_movers_data,
    fetch_synthetic_data,
)
from xai_ranking.scorers import (
    atp_score,
    csrank_score,
    higher_education_score,
    synthetic_equal_score_3ftrs,
)
from xai_ranking.metrics import (
    explanation_sensitivity_all_neighbors,
)

RNG_SEED = 42
N_RUNS = 1

datasets = [
    {
        "name": "Higher Education",
        "data": preprocess_higher_education_data(
            fetch_higher_education_data(year=2020)
        ),
        "scorer": higher_education_score,
        "n_observations": 100,
    },
    {
        "name": "ATP",
        "data": preprocess_atp_data(fetch_atp_data()),
        "scorer": atp_score,
        "n_observations": 86,
    },
    {
        "name": "CSRank",
        "data": preprocess_csrank_data(fetch_csrank_data()),
        "scorer": csrank_score,
        "n_observations": 100,
    },
    {
        "name": "Synthetic_0",
        "data": preprocess_synthetic_data(
            fetch_synthetic_data(synth_dt_version=0, item_num=2000)
        ),
        "scorer": synthetic_equal_score_3ftrs,
        "n_observations": 100,
    },
    {
        "name": "Synthetic_1",
        "data": preprocess_synthetic_data(
            fetch_synthetic_data(synth_dt_version=1, item_num=2000)
        ),
        "scorer": synthetic_equal_score_3ftrs,
        "n_observations": 100,
    },
    {
        "name": "Synthetic_2",
        "data": preprocess_synthetic_data(
            fetch_synthetic_data(synth_dt_version=2, item_num=2000)
        ),
        "scorer": synthetic_equal_score_3ftrs,
        "n_observations": 100,
    },
]

xai_methods = [
    {
        "iterations": 1,
        "name": "LIME",
        "experiment": lime_experiment,
        "type": "score",
        "kwargs": {"mode": "regression"},  # classification, regression
    },
    {
        "iterations": 1,
        "name": "SHAP",
        "experiment": shap_experiment,
        "type": "score",
        "kwargs": {},
    },
    {
        "iterations": 1,
        "name": "ShaRP_RANK",
        "experiment": sharp_experiment,
        "type": "rank",
        "kwargs": {
            "qoi": "rank",
            "verbose": True,
            "sample_size": None,
            "measure": "shapley",
            "n_jobs": -1,
            "replace": False,
        },
    },
    {
        "iterations": 1,
        "name": "ShaRP_SCORE",
        "experiment": sharp_experiment,
        "type": "score",
        "kwargs": {
            "qoi": "rank_score",
            "verbose": True,
            "sample_size": None,
            "measure": "shapley",
            "n_jobs": -1,
            "replace": False,
        },
    },
    # {
    #     "iterations": 1,
    #     "name": "ShaRP_TOPK",
    #     "experiment": sharp_experiment,
    #     "kwargs": {
    #         "qoi": "top_k",
    #         "verbose": True,
    #         "sample_size": None,
    #         "measure": "shapley",
    #         "n_jobs": -1,
    #         "replace": True,
    #     },
    # },
    {
        "iterations": 1,
        "name": "HRE_DT",
        "type": "score",
        "experiment": hierarchical_ranking_explanation,
        "kwargs": {"model_type": "DT", "s": 10},  # DT, LR, OLS, PLS
    },
    {
        "iterations": 1,
        "name": "HRE_LR",
        "type": "score",
        "experiment": hierarchical_ranking_explanation,
        "kwargs": {"model_type": "LR", "s": 10},  # DT, LR, OLS, PLS
    },
    {
        "iterations": 1,
        "name": "HRE_OLS",
        "type": "score",
        "experiment": hierarchical_ranking_explanation,
        "kwargs": {"model_type": "OLS", "s": 10},  # DT, LR, OLS, PLS
    },
    {
        "iterations": 1,
        "name": "HRE_PLS",
        "type": "score",
        "experiment": hierarchical_ranking_explanation,
        "kwargs": {"model_type": "PLS", "s": 10},  # DT, LR, OLS, PLS
    },
    # {
    #     "iterations": 1,
    #     "name": "HIL_Shapley",
    #     "experiment": human_in_the_loop_experiment,
    #     "kwargs": {"upper_bound": 1, "lower_bound": None, "method_type": "shapley"},
    # },
    {
        "iterations": 1,
        "name": "HIL_Standardized-Shapley",
        "type": "std-score",
        "experiment": human_in_the_loop_experiment,
        "kwargs": {
            "upper_bound": 1,
            "lower_bound": None,
            "method_type": "standardized shapley",
        },
    },
    {
        "iterations": 1,
        "name": "HIL_Rank-Shapley",
        "experiment": human_in_the_loop_experiment,
        "type": "std-rank",
        "kwargs": {
            "upper_bound": 1,
            "lower_bound": None,
            "method_type": "rank-relevance shapley",
        },
    },
    # {
    #     "iterations": 1,
    #     "name": "RankLIME",
    #     "experiment": rank_lime_experiment,
    #     "kwargs": {
    #         "explanation_size": 3,
    #         "rank_similarity_coefficient": lambda x, y: kendalltau(x, y)[0],
    #         "individual_masking": True,
    #         "use_entry": 0,
    #         "use_pandas_where": False,
    #     },
    # },
    # {"iterations": 1, "name": "Participation", "experiment": participation_experiment},
]

def read_results_from_files():
    return_dict = {}
    for cur_dataset in datasets:
        return_dict[cur_dataset["name"]] = {}
        for cur_xai_method in xai_methods:
            return_dict[cur_dataset["name"]][cur_xai_method["name"]] = []
            for iteration in range(cur_xai_method["iterations"]):
                fname = (
                    f"results/contributions/_contributions_"
                    f"{cur_dataset['name']}_{cur_xai_method['name']}_"
                    f"{iteration}.csv"
                )
                if os.path.isfile(fname):
                    (
                        return_dict[cur_dataset["name"]][cur_xai_method["name"]].append(
                            pd.read_csv(fname, index_col=0)
                        )
                    )
    return return_dict

results = read_results_from_files()
        

In [9]:
# results

In [13]:
for dataset in datasets:
    rankings = scores_to_ordering(dataset["scorer"](dataset["data"][0]))
    data = dataset["data"][0]
    for method in xai_methods:
        print(dataset["name"], " ", method["name"])
        res = results[dataset["name"]][method["name"]][0]

        result = explanation_sensitivity_all_neighbors(
            data, 
            res,
            rankings,
            measure="euclidean",
            threshold=None,
            normalization=False
        )
        
        euc_dist = []
        rank_dist = []
        feat_dist = []
        for index in range(data.shape[0]):
            measure_distances, rank_distances, feature_distances = result(row_idx=index)
            # print(measure_distances)
            # print(rank_distances)
            euc_dist.append(measure_distances.tolist())
            rank_dist.append(abs(rank_distances).tolist())
            feat_dist.append(feature_distances.tolist())
            # if index == 5:
            #     print(euc_dist)
            #     print(rank_dist)

        # sperman_rank = spearmanr(np.array(euc_dist),np.array(rank_dist))
        # print(sperman_rank.statistic)
        # print(sperman_rank.pvalue)
        # Save explanation distances
        result_fname = (
                f"results/sensitivity/correlations/_explanation_distances_{dataset['name']}_"
                f"{method['name']}.csv"
            )
        result_df = pd.DataFrame(euc_dist)
        result_df.to_csv(result_fname)
        # Save ranks differences once (identical for all methods)
        result_fname = (
                f"results/sensitivity/correlations/_rank_distances_{dataset['name']}.csv"
            )
        if not os.path.isfile(result_fname):
            result_df = pd.DataFrame(rank_dist)
            result_df.to_csv(result_fname)
        # Save feature distances once (identical for all methods)
        result_fname = (
                f"results/sensitivity/correlations/_feature_distances_{dataset['name']}.csv"
            )
        if not os.path.isfile(result_fname):
            result_df = pd.DataFrame(feat_dist)
            result_df.to_csv(result_fname)

ATP   LIME
ATP   SHAP
ATP   ShaRP_RANK
ATP   ShaRP_SCORE
ATP   HRE_DT
ATP   HRE_LR
ATP   HRE_OLS
ATP   HRE_PLS
ATP   HIL_Standardized-Shapley
ATP   HIL_Rank-Shapley
CSRank   LIME
CSRank   SHAP
CSRank   ShaRP_RANK
CSRank   ShaRP_SCORE
CSRank   HRE_DT
CSRank   HRE_LR
CSRank   HRE_OLS
CSRank   HRE_PLS
CSRank   HIL_Standardized-Shapley
CSRank   HIL_Rank-Shapley
Synthetic_0   LIME
Synthetic_0   SHAP
Synthetic_0   ShaRP_RANK
Synthetic_0   ShaRP_SCORE
Synthetic_0   HRE_DT
Synthetic_0   HRE_LR
Synthetic_0   HRE_OLS
Synthetic_0   HRE_PLS
Synthetic_0   HIL_Standardized-Shapley
Synthetic_0   HIL_Rank-Shapley
Synthetic_1   LIME
Synthetic_1   SHAP
Synthetic_1   ShaRP_RANK
Synthetic_1   ShaRP_SCORE
Synthetic_1   HRE_DT
Synthetic_1   HRE_LR
Synthetic_1   HRE_OLS
Synthetic_1   HRE_PLS
Synthetic_1   HIL_Standardized-Shapley
Synthetic_1   HIL_Rank-Shapley
Synthetic_2   LIME
Synthetic_2   SHAP
Synthetic_2   ShaRP_RANK
Synthetic_2   ShaRP_SCORE
Synthetic_2   HRE_DT
Synthetic_2   HRE_LR
Synthetic_2   HRE_O

In [ ]:
sperman_rank = spearmanr(np.array(euc_dist),np.array(rank_dist))
print(sperman_rank.statistic)
print(sperman_rank.pvalue)

In [23]:
def read_corr_results_from_files():
    ranks_dict = {}
    features_dict = {}
    explanations_dict = {}
    for cur_dataset in datasets:
        # Read rank and feature differences from file
        ranks_dict[cur_dataset["name"]] = []
        fname = (
            f"results/sensitivity/correlations/_rank_distances_"
            f"{cur_dataset['name']}.csv"
        )
        if os.path.isfile(fname):
                ranks_dict[cur_dataset["name"]].append(pd.read_csv(fname, index_col=0))
        features_dict[cur_dataset["name"]] = []
        fname = (
            f"results/sensitivity/correlations/_feature_distances_"
            f"{cur_dataset['name']}.csv"
        )
        if os.path.isfile(fname):
                features_dict[cur_dataset["name"]].append(pd.read_csv(fname, index_col=0))

        # Read explanation differences from files
        explanations_dict[cur_dataset["name"]] = {}
        for cur_xai_method in xai_methods:
            explanations_dict[cur_dataset["name"]][cur_xai_method["name"]] = []
            fname = (
                f"results/sensitivity/correlations/_explanation_distances_"
                f"{cur_dataset['name']}_{cur_xai_method['name']}.csv"
            )
            if os.path.isfile(fname):
                (
                    explanations_dict[cur_dataset["name"]][cur_xai_method["name"]].append(
                        pd.read_csv(fname, index_col=0)
                    )
                )
    return ranks_dict, features_dict, explanations_dict



In [24]:
ranks, features, explanations = read_corr_results_from_files()

{'ATP': [           0         1         2         3         4         5         6  \
0   0.000000  0.073287  0.085785  0.109522  0.120125  0.125682  0.177840   
1   0.073287  0.000000  0.042474  0.061351  0.086423  0.106719  0.117346   
2   0.085785  0.042474  0.000000  0.042755  0.079101  0.103706  0.108471   
3   0.109522  0.061351  0.042755  0.000000  0.047339  0.079656  0.083271   
4   0.120125  0.086423  0.079101  0.047339  0.000000  0.040125  0.081994   
..       ...       ...       ...       ...       ...       ...       ...   
81  0.356351  0.324794  0.319655  0.295166  0.265748  0.276051  0.246745   
82  0.328369  0.297767  0.284456  0.255470  0.227473  0.238663  0.218268   
83  0.338814  0.310963  0.301105  0.273141  0.241203  0.248900  0.232598   
84  0.369249  0.340913  0.331919  0.307262  0.278523  0.289166  0.263162   
85  0.342554  0.317855  0.303453  0.273459  0.240967  0.245286  0.240187   

           7         8         9  ...        76        77        78        79 

In [25]:
print(explanations)

{'ATP': {'LIME': [            0          1          2          3          4          5  \
0    0.000000   0.640224   4.735929   7.698893   5.929526  10.557191   
1    0.640224   0.000000   4.399342   7.403870   5.886552  10.766540   
2    4.735929   4.399342   0.000000   3.059633   3.768795   9.686601   
3    7.698893   7.403870   3.059633   0.000000   4.824319  10.062162   
4    5.929526   5.886552   3.768795   4.824319   0.000000   6.054099   
..        ...        ...        ...        ...        ...        ...   
81  30.925759  31.317699  32.007026  32.385686  30.968761  30.070696   
82  33.092618  33.376827  32.475010  31.707755  31.510110  30.517304   
83  31.177201  31.484730  31.110343  30.763015  30.110054  29.304854   
84  31.098221  31.413167  31.273561  31.131791  30.203154  29.443006   
85  30.358775  30.638519  29.624872  28.873027  28.630787  27.504674   

            6          7          8          9  ...         76         77  \
0    7.870428   7.804089  12.069520  11.